<a href="https://colab.research.google.com/github/AdamKimhub/Msproject1/blob/main/forfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    dataset_path = '/content/drive/My Drive/dataset'

    # Install required packages
    !pip install pymatgen torch_geometric mp_api
    import torch
    from torch_geometric.data import Data

else:
    dataset_path = 'dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
from pathlib import Path
import ast
import numpy as np
import pandas as pd
from pymatgen.core import Structure, PeriodicSite, DummySpecie
from pymatgen.analysis.local_env import MinimumDistanceNN

import to_graph

## For Highly Concentrated Defects Datasets

In [28]:
high_dataset = ["high_BN", "high_GaSe", "high_InSe", "high_MoS2", "high_P", "high_WSe2"]
to_merge = [pd.read_csv(f"{dataset_path}/combined/{high_data}.csv") for high_data in high_dataset]

high_df  = pd.concat(to_merge, ignore_index=True)

high_copy = high_df.copy()

In [29]:
high_copy = high_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
high_copy = high_copy.corr()
high_copy

,Unnamed: 0,energy,formation_energy,energy_per_atom,E_1,band_gap_value,vacancy_sites,substitution_sites,defect_sites
Unnamed: 0,1.000000,0.065208,0.543890,0.020091,-0.059335,-0.473371,0.850225,0.861470,0.917964
energy,0.065208,1.000000,-0.375896,0.863569,0.850896,0.421550,-0.140364,-0.149887,-0.155645
formation_energy,0.543890,-0.375896,1.000000,-0.610129,-0.220687,-0.382395,0.526679,0.415944,0.505736
energy_per_atom,0.020091,0.863569,-0.610129,1.000000,0.542292,0.280060,-0.057697,-0.062920,-0.064677
E_1,-0.059335,0.850896,-0.220687,0.542292,1.000000,0.500377,-0.317357,-0.320360,-0.342003
band_gap_value,-0.473371,0.421550,-0.382395,0.280060,0.500377,1.000000,-0.621014,-0.497336,-0.600002
vacancy_sites,0.850225,-0.140364,0.526679,-0.057697,-0.317357,-0.621014,1.000000,0.738407,0.932796
substitution_sites,0.861470,-0.149887,0.415944,-0.062920,-0.320360,-0.497336,0.738407,1.000000,0.931824
defect_sites,0.917964,-0.155645,0.505736,-0.064677,-0.342003,-0.600002,0.932796,0.931824,1.000000


## For Lowly Concentrated Defects Datasets

In [30]:
low_dataset = ["low_MoS2", "low_WSe2"]
to_merge = [pd.read_csv(f"{dataset_path}/combined/{low_data}.csv") for low_data in low_dataset]

low_df  = pd.concat(to_merge, ignore_index=True)

low_copy = low_df.copy()

In [31]:
low_copy = low_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
low_copy = low_copy.corr()
low_copy

,Unnamed: 0,energy,energy_per_atom,E_1,formation_energy,band_gap_value,vacancy_sites,substitution_sites,defect_sites
Unnamed: 0,1.000000,0.086043,0.035531,0.005665,0.101248,-0.081309,0.116035,-0.094577,0.122049
energy,0.086043,1.000000,0.866313,0.685840,0.692029,-0.777393,0.748926,-0.735307,0.077459
energy_per_atom,0.035531,0.866313,1.000000,0.741425,0.326091,-0.576637,0.317820,-0.309983,0.044570
E_1,0.005665,0.685840,0.741425,1.000000,0.264414,-0.576634,0.319776,-0.327534,-0.044128
formation_energy,0.101248,0.692029,0.326091,0.264414,1.000000,-0.857472,0.881046,-0.865856,0.086395
band_gap_value,-0.081309,-0.777393,-0.576637,-0.576634,-0.857472,1.000000,-0.710278,0.694966,-0.087088
vacancy_sites,0.116035,0.748926,0.317820,0.319776,0.881046,-0.710278,1.000000,-0.984543,0.087911
substitution_sites,-0.094577,-0.735307,-0.309983,-0.327534,-0.865856,0.694966,-0.984543,1.000000,0.087911
defect_sites,0.122049,0.077459,0.044570,-0.044128,0.086395,-0.087088,0.087911,0.087911,1.000000


## For High and Low Concentrations of Defects Datasets

In [41]:
# Read file
comb_df = pd.read_csv(f"{dataset_path}/combined/combined.csv")

In [42]:
comb_copy = comb_df.copy()
comb_copy = comb_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
comb_copy = comb_copy.corr()
comb_copy

,Unnamed: 0,energy,formation_energy,energy_per_atom,E_1,band_gap_value,vacancy_sites,substitution_sites,defect_sites
Unnamed: 0,1.000000,-0.543755,-0.459145,-0.348285,0.060681,0.075284,-0.455989,-0.486571,-0.513516
energy,-0.543755,1.000000,0.291578,0.876757,0.498878,-0.065589,0.500522,0.481216,0.534796
formation_energy,-0.459145,0.291578,1.000000,-0.058278,0.188604,-0.450856,0.756871,0.524373,0.697688
energy_per_atom,-0.348285,0.876757,-0.058278,1.000000,0.388663,-0.027159,0.351724,0.344729,0.379397
E_1,0.060681,0.498878,0.188604,0.388663,1.000000,-0.427162,0.226652,0.048636,0.149743
band_gap_value,0.075284,-0.065589,-0.450856,-0.027159,-0.427162,1.000000,-0.470108,-0.017354,-0.264983
vacancy_sites,-0.455989,0.500522,0.756871,0.351724,0.226652,-0.470108,1.000000,0.684788,0.917422
substitution_sites,-0.486571,0.481216,0.524373,0.344729,0.048636,-0.017354,0.684788,1.000000,0.918218
defect_sites,-0.513516,0.534796,0.697688,0.379397,0.149743,-0.264983,0.917422,0.918218,1.000000


## Data to graphs

In [43]:
# Add strata
datsets = ["high_BN", "high_GaSe", "high_InSe", "high_MoS2", "high_P", "high_WSe2","low_MoS2", "low_WSe2"]

ref_sites_dict = {}

for dataset in datsets:
    mat_split = dataset.split('_')

    # The host material
    the_base = mat_split[1]

    # Get reference structure
    ref_unit_cell = Structure.from_file(f"{dataset_path}/{dataset}/{the_base}.cif")
    cell_source = pd.read_csv(f"{dataset_path}/initial_structures.csv")
    the_cell = cell_source.loc[cell_source["base"] == the_base, "cell_size"].iloc[0]
    reference_structure = ref_unit_cell.make_supercell(ast.literal_eval(the_cell))

    # Get dictionary of type of dataset as keys
    # and number of sites in the reference structure as values
    ref_sites_dict[dataset] = reference_structure.num_sites


def get_conc(row, ref_sites_dict):
    total_num_sites = ref_sites_dict[row["dataset_material"]]

    # Get defect concentration
    defect_conc = round(row["defect_sites"]/total_num_sites,5)
    row["defect_concentration"] = defect_conc

    # The strata column will be in the form of material type_defect concentration
    row["to_strata"] = f"{row['base']}_{row['defect_concentration']}"
    return row

comb_df = comb_df.apply(lambda row: get_conc(row,ref_sites_dict), axis=1)

# Convert the values in the strata to integers to split effectively
unique_values = pd.unique(comb_df["to_strata"])
mapping = {value: i for i, value in enumerate(unique_values)}

comb_df["strata"] = comb_df["to_strata"].map(mapping)

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3104: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3104: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3104: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3104: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issue

In [44]:
# Split the data
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(comb_df, test_size=0.15, stratify=comb_df['strata'], random_state=42)
train_set, val_set = train_test_split(test_set, test_size=0.3, random_state=42)

In [ ]:
def graphy(row):
    # Get the defective structure
    defective_file_path = Path(f"{dataset_path}/{row['dataset_material']}/cifs/{row['_id']}.cif")
    defective_structure = Structure.from_file(defective_file_path)

    # Get the reference(pristine) structure
    ref_file_path = Path(f"{dataset_path}/{row['dataset_material']}/{row['base']}.cif")
    ref_unit_cell = Structure.from_file(ref_file_path)
    the_cell = ast.literal_eval(row['cell'])
    reference_structure = ref_unit_cell.make_supercell(the_cell)

    # Compare the defective and pristine structure to extract the defect sites
    defects_structure = to_graph.get_defects_structure(defective_structure, reference_structure)

    # Convert the defect structure into a crystal graph
    the_nodes, the_edges, the_edge_features = to_graph.get_nodes_edges(defects_structure)

    # Global attributes
    global_attributes = to_graph.get_global_attributes(row)

    # Get the target values
    target_features = [row["band_gap_value"]]

    the_data = Data(x=torch.tensor(the_nodes, dtype=torch.float),
                    edge_index=torch.tensor(the_edges, dtype=torch.long),
                    edge_attr=torch.tensor(the_edge_features, dtype=torch.float),
                    u= torch.tensor(global_attributes, dtype=torch.float),
                    y=torch.tensor(target_features, dtype=torch.float).unsqueeze(0))
    return the_data


# Turn each dataset into graph data and save it
training = train_set.apply(lambda row: graphy(row), axis = 1).tolist()
torch.save(training, f"{dataset_path}/combined/training.pt")

validating = val_set.apply(lambda row: graphy(row), axis = 1).tolist()
torch.save(validating, f"{dataset_path}/combined/validating.pt")

testing = test_set.apply(lambda row: graphy(row), axis = 1).tolist()
torch.save(testing, f"{dataset_path}/combined/testing.pt")